# Trust in Multi-Agent Systems
Welcome to our lesson on trust and explainability in AI! 

# Explainability
As discussed in lecture, we know that _explaining_ unexpected behaviors from our robot can increase trust. In this module, we'll see how this comes into play. Recall our human-aware planning problem from lecture:

## Human-Aware Planning
Recall the Human-Aware Planning problem described in [1](https://arxiv.org/pdf/2105.01220.pdf).

**Input:**

$\mathcal{M}^R$, the robot's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^R, \mathcal{I}^R, \mathcal{G}^R\rangle$, where $\mathcal{D}^R$ is the domain,  $\mathcal{I}^R$ is the initial state, and $\mathcal{G}^R$ is the goal state.

$\mathcal{M}^G$, the human's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^H, \mathcal{I}^H, \mathcal{G}^H\rangle$, where $\mathcal{D}^H$ is the domain,  $\mathcal{I}^H$ is the initial state, and $\mathcal{G}^H$ is the goal state.

## Problem 1: Modeling Incomplete Knowledge
Because we're so familiar with PDDL, we will model both of these problems in PDDL! Check out the references from `ps-03` if you've forgotten how to use it. **TODO: describe this stuff in more detail.**

We'll start with a simplified version of the [Wumpus World](http://users.cecs.anu.edu.au/~patrik/pddlman/wumpus.html). Here, our robot is trying to navigate in a 3x3 grid to pick up a block. The robot knows that its shortest path is unencumbered, but the human has no idea--they think that there's piles of trash on the shortest path to the robot.

![Wumpus World](/res/wumpus-world.png)